In [1]:
# !python3 -m spacy download en
# !python -m spacy download en
# !python -m spacy download en_core_web_sm
# !pip3 install gensim
# !python -m pip install -U gensim
# !pip install gensim
import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from nltk.corpus import stopwords
import spacy
import scipy  
from scipy import spatial
from scipy.spatial.distance import pdist, squareform 
spacy_nlp = spacy.load('en_core_web_sm')
import gensim
import os
import re
from gensim.models.wrappers import LdaMallet
import unicodedata
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import torch.multiprocessing as mp
stemmer = SnowballStemmer("english")

In [3]:
cmp_desc = pd.read_csv('Company Descriptions.csv')
display(cmp_desc.head(), cmp_desc.shape)

,company_name,company_short_description,company_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ..."
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...


(19965, 3)

In [4]:
# First 1000 Instances
cmp_desc = cmp_desc.head(1000)
cmp_desc = cmp_desc.reset_index(drop=True)

In [5]:
cmp_desc.head(1)

,company_name,company_short_description,company_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...


In [6]:
def white_space(string):
    return " ".join(string.split())

In [7]:
for i in range(cmp_desc.shape[0]):
#     print(cmp_desc['company_description'][i])
    cmp_desc['company_description'][i] = white_space(str(cmp_desc['company_description'][i]))
    cmp_desc['company_short_description'][i] = white_space(str(cmp_desc['company_short_description'][i]))

In [8]:
cmp_desc['company_description'].head()

0    Codementor provides live 1:1 help for software...
1    AgShift solution blends Deep Learning with Com...
2    Shipsi empowers any retailer with the ability ...
3    We're helping a global network of developers, ...
4    Biobot Analytics analyzes sewage to estimate o...
Name: company_description, dtype: object

In [9]:
cmp_desc.shape

(1000, 3)

In [10]:
# Data Cleaning

cmp_desc['len_text_company_description'] = None
cmp_desc['len_text_company_short_description'] = None

for i in range(cmp_desc.shape[0]):
    cmp_desc['len_text_company_description'][i] = len(str(cmp_desc['company_description'][i]).split())
    cmp_desc['len_text_company_short_description'][i] = len(str(cmp_desc['company_short_description'][i]).split())

In [11]:
cmp_desc['len_text_company_description'].max()

432

In [12]:
cmp_desc['len_text_company_description'].min()

1

In [13]:
cmp_desc.head()

,company_name,company_short_description,company_description,len_text_company_description,len_text_company_short_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...,73,20
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...,98,13
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...,49,18
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ...",42,18
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...,92,9


In [14]:
cmp_desc['final_company_description'] = None
for i in range(cmp_desc.shape[0]):
    if (cmp_desc['len_text_company_description'][i] >= cmp_desc['len_text_company_short_description'][i]):
        cmp_desc['final_company_description'][i] = cmp_desc['company_description'][i]
    elif cmp_desc['len_text_company_description'][i] < cmp_desc['len_text_company_short_description'][i]:
        cmp_desc['final_company_description'][i] = cmp_desc['company_short_description'][i]

In [15]:
display(cmp_desc.head(), cmp_desc.shape)

,company_name,company_short_description,company_description,len_text_company_description,len_text_company_short_description,final_company_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...,73,20,Codementor provides live 1:1 help for software...
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...,98,13,AgShift solution blends Deep Learning with Com...
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...,49,18,Shipsi empowers any retailer with the ability ...
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ...",42,18,"We're helping a global network of developers, ..."
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...,92,9,Biobot Analytics analyzes sewage to estimate o...


(1000, 6)

In [16]:
def remove_html(text):
    new_text = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", text)
    return(new_text)


# Removing the Accented Chars
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


# Removing Special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-z0-9\s*.]' 
    text = re.sub(pattern, '', text)
    
    return text

# Removing the stop words
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS
def remove_stopwords(text):
    doc = spacy_nlp(text)
    text = [token.text for token in doc if not token.is_stop]
    text = ' '.join(text)
    pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    text = pattern.sub('', text)
    
    return text

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def lemma_preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [17]:
def wrangle_data(doc, url_removal = False, accented_removal = False,
                 special_char = False, lower_doc = False, lemma_words = False, stopwords_removal = False):
    
    counter=0

    counter = counter+1

    # Removing URL's
    if url_removal:
        doc = remove_html(doc)

    # Removing accented Characters
    if accented_removal:
        doc = remove_accented_chars(doc)

    # Lowering the words
    if lower_doc:
        doc = doc.lower()
        doc = doc.replace('.', ' . ', doc.count('.')).replace(',', ' , ', doc.count(','))


    # Removing Special Character
    if special_char:
        doc = doc.replace('$', '')
        doc = doc.replace('%', '')
        doc = doc.replace('\t', '')
        doc = re.sub('\s+', ' ', doc).strip()

        # insert spaces between special characters to isolate them    
        doc = remove_special_characters(doc)

        # Remove extra whitespaces
        doc = re.sub(' +', ' ', doc)

    #  Removing Stop-words
    if stopwords_removal:
        doc = remove_stopwords(doc)


    # Lemmantisation 
    if lemma_words:
        doc = lemma_preprocess(doc)

    return doc

In [18]:
df_list_pos = [i for i in range(cmp_desc.shape[0])]
print(len(df_list_pos))

1000


In [19]:
# Multi-processing

cmp_desc['wrangled_description'] = None

def get_position(doc_num):
    pos_value=[]
    pos_value.append(wrangle_data(cmp_desc['final_company_description'][doc_num], url_removal=True, \
                                    lower_doc=True, stopwords_removal = True, accented_removal = True,\
                                    lemma_words=True))
    return pos_value 

pool = mp.Pool(processes=4)
pos_result_collab = pool.map(get_position, (df_list_pos))
pos_result_collab = [item for sublist in pos_result_collab for item in sublist]
cmp_desc['wrangled_description'] = pos_result_collab
pool.close()

In [20]:
cmp_desc.head()

,company_name,company_short_description,company_description,len_text_company_description,len_text_company_short_description,final_company_description,wrangled_description
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...,73,20,Codementor provides live 1:1 help for software...,"[codementor, provid, live, help, softwar, deve..."
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...,98,13,AgShift solution blends Deep Learning with Com...,"[agshift, solut, blend, deep, learn, vision, a..."
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...,49,18,Shipsi empowers any retailer with the ability ...,"[shipsi, empow, retail, abil, offer, easi, ins..."
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ...",42,18,"We're helping a global network of developers, ...","[help, global, network, develop, journalist, m..."
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...,92,9,Biobot Analytics analyzes sewage to estimate o...,"[biobot, analyt, analyz, sewag, estim, opioid,..."


In [21]:
wrangled_docs = cmp_desc['wrangled_description'].to_list()
print(len(wrangled_docs))
print(wrangled_docs[:2])

1000
[['codementor', 'provid', 'live', 'help', 'softwar', 'develop', 'make', 'easi', 'develop', 'connect', 'expert', 'screen', 'share', 'video', 'chat', 'way', 'help', 'demand', 'live', 'expert', 'help', 'long', 'term', 'dedic', 'mentorship', 'build', 'project', 'faster', 'demand', 'help', 'topic', 'includ', 'rubi', 'python', 'javascript', 'html', 'swift', 'codementor', 'help', 'overcom', 'challeng', 'time', 'advic', 'speed', 'develop', 'process'], ['agshift', 'solut', 'blend', 'deep', 'learn', 'vision', 'autonom', 'inspect', 'produc', 'commod', 'defect', 'qualiti', 'assess', 'make', 'judgement', 'usda', 'specif', 'organ', 'specif', 'patent', 'deep', 'learn', 'model', 'analyz', 'defect', 'sampl', 'imag', 'predict', 'overal', 'qualiti', 'sampl', 'platform', 'reli', 'curat', 'extens', 'real', 'world', 'imag', 'data', 'set', 'teach', 'softwar', 'analyz', 'defect', 'high', 'consist', 'accuraci', 'time', 'solut', 'augment', 'manual', 'inspect', 'provid', 'object', 'consist', 'standard', 'qu

# Step 3: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass processed_docs to gensim.corpora.Dictionary() and call it 'dictionary'.

In [22]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(wrangled_docs)

In [23]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 advic
1 build
2 challeng
3 chat
4 codementor
5 connect
6 dedic
7 demand
8 develop
9 easi
10 expert


In [24]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in wrangled_docs]

In [25]:
# bow_corpus

In [26]:
'''
Preview BOW for our sample preprocessed document
'''
document_num = 1
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 18 ("make") appears 1 time.
Word 23 ("provid") appears 1 time.
Word 28 ("softwar") appears 1 time.
Word 32 ("time") appears 2 time.
Word 36 ("accuraci") appears 1 time.
Word 37 ("agshift") appears 1 time.
Word 38 ("analyz") appears 2 time.
Word 39 ("assess") appears 1 time.
Word 40 ("augment") appears 1 time.
Word 41 ("autonom") appears 1 time.
Word 42 ("blend") appears 1 time.
Word 43 ("chain") appears 1 time.
Word 44 ("commod") appears 1 time.
Word 45 ("consist") appears 2 time.
Word 46 ("curat") appears 1 time.
Word 47 ("data") appears 1 time.
Word 48 ("deep") appears 2 time.
Word 49 ("defect") appears 3 time.
Word 50 ("extens") appears 1 time.
Word 51 ("high") appears 1 time.
Word 52 ("imag") appears 2 time.
Word 53 ("inspect") appears 2 time.
Word 54 ("interpret") appears 1 time.
Word 55 ("judgement") appears 1 time.
Word 56 ("learn") appears 2 time.
Word 57 ("manual") appears 1 time.
Word 58 ("model") appears 1 time.
Word 59 ("object") appears 1 time.
Word 60 ("organ") appea

In [27]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''
num_topics = 27 # Specified
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = num_topics, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [28]:
topic_list = []

for i in range(num_topics):
    inter_topic_list = []
    for j in range(10):
        inter_topic_list.append(lda_model.show_topic(i, 10)[j][0])
    topic_list.append(inter_topic_list)

print(topic_list)

[['compani', 'provid', 'technolog', 'busi', 'data', 'servic', 'product', 'develop', 'platform', 'base'], ['product', 'compani', 'custom', 'properti', 'provid', 'servic', 'base', 'technolog', 'user', 'media'], ['compani', 'product', 'servic', 'base', 'network', 'technolog', 'solut', 'provid', 'system', 'time'], ['provid', 'network', 'compani', 'care', 'health', 'onlin', 'platform', 'improv', 'xfire', 'high'], ['compani', 'media', 'restaur', 'build', 'platform', 'brand', 'digit', 'industri', 'technolog', 'time'], ['data', 'compani', 'provid', 'platform', 'solut', 'product', 'custom', 'softwar', 'manag', 'market'], ['credit', 'compani', 'platform', 'consum', 'design', 'offer', 'help', 'financ', 'custom', 'financi'], ['patient', 'deliveri', 'robot', 'diseas', 'medic', 'technolog', 'irobot', 'access', 'provid', 'medicin'], ['user', 'question', 'peopl', 'answer', 'compani', 'social', 'interest', 'process', 'share', 'help'], ['compani', 'digit', 'develop', 'busi', 'product', 'technolog', 'par

In [29]:
unseen_document = cmp_desc['wrangled_description'][2]
print(unseen_document)

['shipsi', 'empow', 'retail', 'abil', 'offer', 'easi', 'instant', 'ship', 'option', 'checkout', 'pagewithout', 'worri', 'logist', 'exist', 'infrastructur', 'mile', 'deliveri', 'network', 'help', 'custom', 'order', 'shipsi', 'found', 'headquart', 'california']


In [30]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(unseen_document)

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print('Topic Number:', index)
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Topic Number: 5
Score: 0.6079071164131165	 Topic: 0.014*"data" + 0.012*"compani" + 0.010*"provid" + 0.010*"platform" + 0.010*"solut" + 0.009*"product" + 0.008*"custom" + 0.008*"softwar" + 0.008*"manag" + 0.008*"market"
Topic Number: 25
Score: 0.35647183656692505	 Topic: 0.016*"video" + 0.012*"compani" + 0.008*"ventur" + 0.007*"creat" + 0.007*"peopl" + 0.007*"health" + 0.007*"team" + 0.007*"share" + 0.007*"technolog" + 0.006*"platform"


In [31]:
topic_list[6]

['credit',
 'compani',
 'platform',
 'consum',
 'design',
 'offer',
 'help',
 'financ',
 'custom',
 'financi']

In [32]:
cmp_desc['score'] = None
cmp_desc['Topic_number'] = None
cmp_desc['Topics_label'] = None

In [33]:
for i in range(cmp_desc.shape[0]):
    unseen_document = cmp_desc['wrangled_description'][i]
    bow_vector = dictionary.doc2bow(unseen_document)
    
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        cmp_desc['score'][i] = score
        cmp_desc['Topic_number'][i] = index
        cmp_desc['Topics_label'][i] = topic_list[index]

In [34]:
cmp_desc.columns

Index(['company_name', 'company_short_description', 'company_description',
       'len_text_company_description', 'len_text_company_short_description',
       'final_company_description', 'wrangled_description', 'score',
       'Topic_number', 'Topics_label'],
      dtype='object')

In [35]:
cmp_desc = cmp_desc[['company_name', 'company_short_description', 'company_description',
       'final_company_description', 'wrangled_description', 'score',
       'Topic_number', 'Topics_label']]

In [36]:
cmp_desc.head()

,company_name,company_short_description,company_description,final_company_description,wrangled_description,score,Topic_number,Topics_label
0,Codementor,Codementor is an online marketplace connecting...,Codementor provides live 1:1 help for software...,Codementor provides live 1:1 help for software...,"[codementor, provid, live, help, softwar, deve...",0.115458,25,"[video, compani, ventur, creat, peopl, health,..."
1,AgShift,AgShift is designing world's most advanced aut...,AgShift solution blends Deep Learning with Com...,AgShift solution blends Deep Learning with Com...,"[agshift, solut, blend, deep, learn, vision, a...",0.325258,10,"[data, custom, solut, servic, provid, wireless..."
2,Shipsi,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...,Shipsi empowers any retailer with the ability ...,"[shipsi, empow, retail, abil, offer, easi, ins...",0.356472,25,"[video, compani, ventur, creat, peopl, health,..."
3,OpenNews,"OpenNews helps a global network of developers,...","We're helping a global network of developers, ...","We're helping a global network of developers, ...","[help, global, network, develop, journalist, m...",0.961475,24,"[platform, compani, data, market, custom, netw..."
4,Biobot Analytics,Biobot Analytics analyzes city sewage to estim...,Biobot Analytics analyzes sewage to estimate o...,Biobot Analytics analyzes sewage to estimate o...,"[biobot, analyt, analyz, sewag, estim, opioid,...",0.982163,0,"[compani, provid, technolog, busi, data, servi..."
